## Exercise 3.2

In [30]:
from itertools import combinations
from scipy.special import comb, perm
import numpy as np


def prob_all_colors_drawn(m, N):
    ''' 
    m is number of balls drawn
    N is a list containing how many of each color is in the urn 
    '''
    k = len(N) # number of colors
    
    total = 1.0 # start with 1
    for i in range(1,k+1):

        # calculate each sum term
        conjunction_prob = 0
        for Ns in combinations(N, i): # for all combinations of i colors
            conjunction_prob += comb(sum(N) - sum(Ns), m,True)/comb(sum(N),m,True)

        # alternately add or subtract the sum term 
        total += ((-1)**i)*conjunction_prob
    return total

N = [10,10,10,10,10] # define the urn
for i in range(40):
    print("Draw {:2d} balls, prob = {:.5f}".format(i, prob_all_colors_drawn(i, N)))

Draw  0 balls, prob = 0.00000
Draw  1 balls, prob = -0.00000
Draw  2 balls, prob = -0.00000
Draw  3 balls, prob = -0.00000
Draw  4 balls, prob = -0.00000
Draw  5 balls, prob = 0.04720
Draw  6 balls, prob = 0.14159
Draw  7 balls, prob = 0.26280
Draw  8 balls, prob = 0.39045
Draw  9 balls, prob = 0.51074
Draw 10 balls, prob = 0.61647
Draw 11 balls, prob = 0.70513
Draw 12 balls, prob = 0.77701
Draw 13 balls, prob = 0.83382
Draw 14 balls, prob = 0.87783
Draw 15 balls, prob = 0.91133
Draw 16 balls, prob = 0.93647
Draw 17 balls, prob = 0.95507
Draw 18 balls, prob = 0.96865
Draw 19 balls, prob = 0.97843
Draw 20 balls, prob = 0.98538
Draw 21 balls, prob = 0.99025
Draw 22 balls, prob = 0.99361
Draw 23 balls, prob = 0.99589
Draw 24 balls, prob = 0.99741
Draw 25 balls, prob = 0.99841
Draw 26 balls, prob = 0.99905
Draw 27 balls, prob = 0.99944
Draw 28 balls, prob = 0.99969
Draw 29 balls, prob = 0.99983
Draw 30 balls, prob = 0.99991
Draw 31 balls, prob = 0.99996
Draw 32 balls, prob = 0.99998
Draw 3

## Monte Carlo approximation

We can verify the correctness of our solution above by simulating drawing from an urn and see if the frequency of getting all colors is the same as the probability given above.

In [41]:
import numpy as np

def draw_from_urn(m,N):
    ''' draws m balls from the urn '''
    N = N.copy()
    out = np.array([0]*len(N))
    for i in range(m):
        choice = np.random.choice(len(N), p=N/np.sum(N))
        out[choice] += 1
        N[choice] -= 1
    return out

In [32]:
n = 5000 # number of simulations

num_all = 0
for i in range(n):
    if(np.all(draw_from_urn(7,[10,10,10,10,10]) > 0)):  # if there is at least one of each color
        num_all += 1

print("Approximate probability of drawing all colors after 7 draws: ", num_all/n)
print("Exact probability: ", prob_all_colors_drawn(7, [10,10,10,10,10]))



Approximate probability of drawing all colors after 7 draws:  0.2618
Exact probability:  0.26280380119418056


## Exercise 3.3

In [33]:
from itertools import combinations
from scipy.special import comb, perm
import numpy as np


def prob_3_colors_drawn(m, N):
    ''' 
    This function calculates p(~A1,~A2,~A3,A4,A5...Ak| k, Ns)
    m is number of balls drawn
    N is a list containing how many of each color is in the urn 
    '''
    k = len(N) # number of colors
    N = np.array(N)
    
    total = 1 # start with 1
    for i in range(1,4):

        # calculate each sum term
        conjunction_prob = 0
        for Ns in combinations(N[:3], i): # for all combinations of i colors
            conjunction_prob += comb(sum(N) - sum(Ns)-sum(N[3:]), m)/comb(sum(N)-sum(N[3:]),m)
            #print(Ns, conjunction_prob)

        # alternately add or subtract the sum term 
        total += ((-1)**(i))*conjunction_prob
    
    # prob(A_4, A_5, ...)
    total *= comb(sum(N) - sum(N[3:]), m)/comb(sum(N),m)
    return total


print("lower bound on p(colors=3|k=3) = {:.5f}\n".format(prob_3_colors_drawn(20, [48,1,1])))

posterior_upper_bound = np.zeros((40,))
for k in range(3,40):
    N = [int((50-k)/3)]*3 + [1]*(k-3)
    posterior_upper_bound[k] = 6.66*comb(len(N),3)*prob_3_colors_drawn(20, N)
    #print("N = ", N)
    #print("k ={:2d}, upper bound on prob = {:.5f}\n".format(k, posterior_upper_bound[k]))

lower bound on p(colors=3|k=3) = 0.15510



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars


In [34]:
posterior_upper_bound = np.nan_to_num(posterior_upper_bound)
np.set_printoptions(precision=3, suppress=True)

print("Full upper bound on posterior over k:")
print(posterior_upper_bound,"\n")

print("Upper bound on probability that k > 20:")
print(sum(posterior_upper_bound[21:]))

Full upper bound on posterior over k:
[ 0.     0.     0.     6.66  15.057 21.624 21.589 21.355 19.628 12.164
  9.822  7.758  3.378  2.482  1.815  0.548  0.381  0.266  0.052  0.035
  0.023  0.002  0.002  0.001  0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ] 

Upper bound on probability that k > 20:
0.005271749464451704


The above upper bound on the probability that $k > 20$ allows us to say that with at least 99% confidence that $3 \le k \le 20$. 


## Monte carlo again
We confirm the quality of the function above by comparing it to a monte carlo estimate.

In [35]:
n = 10000 # number of simulations
N = [15,15,15,1,1]

num_three = 0
for i in range(n):
    out = draw_from_urn(20,N) > 0
    if(np.all(out[:3]) and not np.any(out[3:])):  # if there is at least one of each color
        num_three += 1

print("Estimate: ", num_three/n)

print("True answer: ", prob_3_colors_drawn(20, N))



Estimate:  0.3218
True answer:  0.32469011964911704


## 2nd Alternative Solution
This solution calculates the exact likelihood instead of an upper bound on it (equations derived in Alternative Solution), and uses it to tighten the upper bound on the posterior.



In [36]:
from itertools import combinations
from scipy.special import comb, perm
import numpy as np

def prob_any_3_colors_drawn(m, N): # probability that any m colors are drawn
    k = len(N)
    N = np.array(N)

    total = 0.0
    for i in [1,2,3]:
        conjunction_prob = 0
        for Ns in combinations(N, i):
            conjunction_prob += comb(k-i, 3-i)*comb(sum(Ns), m)
        total += ((-1)**(i-1))*conjunction_prob
    total /= comb(sum(N), m)
    return total

posterior_upper_bound = np.zeros((40,))
for k in range(3,40):
    N = [int((50-k)/3)]*3 + [1]*(k-3)
    posterior_upper_bound[k] = (6.66*prob_any_3_colors_drawn(20, N))

Shown below is the upper bound on the probability that $k > 13$. Hence we can be more than 99% sure that $3 \ge k \ge 13$.

In [37]:
np.set_printoptions(precision=3, suppress=True)
print("Full upper bound on posterior over k:")
print(posterior_upper_bound,"\n")
print("Upper bound on probability that k > 13:")
print(sum(posterior_upper_bound[14:])) # upper bound on the probability that k >= 14

Full upper bound on posterior over k:
[0.    0.    0.    6.66  3.764 2.163 1.08  0.61  0.351 0.145 0.082 0.047
 0.015 0.009 0.005 0.001 0.001 0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.   ] 

Upper bound on probability that k > 13:
0.007385411327220326


We confirm the quality of the function above by comparing it to a monte carlo estimate.

In [ ]:
n = 10000 # number of simulations
N = [1,5,5,14,1]

num_three = 0
for i in range(n):
    out = draw_from_urn(20,N) > 0
    if sum(out > 0) == 3:
        num_three += 1

print("Estimate: ", num_three/n)

print("True answer: ", prob_any_3_colors_drawn(20, N))



Estimate:  0.0461
True answer:  0.04617122008426356
